# Document Scanner
* Bogdan 
* Adrian 
* Andrew

In [8]:
import cv2 as cv
import numpy as np
from matplotlib import pyplot as plt 
import matplotlib.cm as cm
from easygui import *


def selectImage():
    
    # select image from file
    try:
        path = fileopenbox()
    except:
        print("Failed to select file")
        
    return cv.imread(path)



def getContours(img,original):
    
    # accessing contours on image
    try:
        contours,hierarchy = cv.findContours(img,cv.RETR_EXTERNAL,cv.CHAIN_APPROX_NONE)
        border = np.zeros_like(original)
        corners = np.zeros_like(original)
    
        contours = sorted(contours, key=cv.contourArea, reverse=True)
        biggest_contour = contours[0]
    
        cv.drawContours(border, biggest_contour, -1, (255, 0, 0), 3)
    except:
        print("Failed to draw Contours")

        
    try:
        perimeter = cv.arcLength(biggest_contour,True)
        approx = cv.approxPolyDP(biggest_contour,0.02*perimeter,True)
        objCor = len(approx)
    except:
        print("Failed to allocate perimeter, approx and objCor values")
        
    # error checking for 
    for corner in approx:
        x = int(corner[0][0])
        y = int(corner[0][1])
        cv.circle(corners,(x,y),10,(255,0,0),cv.FILLED)
         
    return approx, border, corners



def formatPoints(pts):
    
    try:
        pt1 = pts[0][0]
        pt2 = pts[1][0]  
        pt3 = pts[2][0]
        pt4 = pts[3][0]
    except:
        print("Failed to set points")
    
    return [pt1,pt2,pt3,pt4]



def getDocumentCorners(img):
    try:
        gray = cv.cvtColor(img,cv.COLOR_BGR2GRAY)

        # Thresholding
        thresh1 = cv.inRange(gray, 155, 255)

        # Morphing the thresholded mask
        shape = cv.getStructuringElement(cv.MORPH_ELLIPSE,(7,7))
    except:
        print("Failed to create threshold")
        
    try:
        mask = cv.morphologyEx(thresh1,cv.MORPH_CLOSE,shape)
        mask = cv.morphologyEx(mask,cv.MORPH_OPEN,shape)
    except:
        print("Failed to mask image")
    
    # Detecting the contours and corners
    try:
        cornerPoints, border, corners = getContours(mask,img)    
        cornerPoints = formatPoints(cornerPoints)        
    except:
        print("Failed to run getContours")
    
    return cornerPoints, thresh1, mask, border, corners, gray



def sortPoints(points):
    
    try:
        sorted_pts = np.zeros((4, 2), dtype="float32")
        s = np.sum(points, axis=1)
        diff = np.diff(points, axis=1)
        unique = np.unique(s)
        print("Sums",s)
    except:
        print("Failed to sort points")
        
    try:
        # Checks if there are duplicate sums
        if not len(unique) == len(s):
            largest = points[0]
            smallest = points[0]

            for i in range(1,len(points)):
                if np.sum(points[i]) == np.sum(largest):
                    # If there are two largest sums that are the same
                    # the point with the highest x value id chosen as the 3rd point
                    # The other is chosen as the last point
                    if points[i][0] > largest[0]:
                        sorted_pts[2] = points[i]
                        sorted_pts[3] = largest
                    else:
                        sorted_pts[2] = largest
                        sorted_pts[3] = points[i]

                if np.sum(points[i]) > np.sum(largest):
                    largest = points[i]

                if np.sum(points[i]) == np.sum(smallest):
                    # If there are two smallest sums that are the same
                    # the point with the smallest x value id chosen as the 1st point
                    # The other is chosen as the 2nd point
                    if points[i][0] < smallest[0]:
                        sorted_pts[0] = points[i]
                        sorted_pts[1] = smallest
                    else:
                        sorted_pts[0] = smallest
                        sorted_pts[1] = points[i]

                if np.sum(points[i]) < np.sum(smallest):
                    smallest = points[i]
        else:
            sorted_pts[0] = points[np.argmin(s)]
            sorted_pts[2] = points[np.argmax(s)]

            sorted_pts[1] = points[np.argmin(diff)]
            sorted_pts[3] = points[np.argmax(diff)]
    except:
        print("Failed to organise points")
        
    try:
        # Fills in the unassigned values of the sorted points
        if 0 in sorted_pts:
            result = np.where(sorted_pts == 0)
            for index in np.unique(result[0]):
                if index == 0:
                    sorted_pts[0] = points[np.argmin(s)]
                elif index == 1:
                    sorted_pts[1] = points[np.argmin(diff)]
                elif index == 2:
                    sorted_pts[2] = points[np.argmax(s)]
                elif index == 3:
                    sorted_pts[3] = points[np.argmax(diff)]
    except: 
        printf("Failed to assign point values")
        
    return sorted_pts


def warpPerspective(img,pts):
    
    try:
        #Calculate transformation matrix
        height, width = img.shape[:2]

        #Points on the document corners
        pts1 = np.float32(pts)

        #Points of the warped image 
        pts2 = np.float32([[0, 0], [width, 0], [width, height], [0, height]])

        #Transform image
        transformation_matrix = cv.getPerspectiveTransform(pts1, pts2)
        warped = cv.warpPerspective(img, transformation_matrix, (width, height))
        
    except:
        printf("Failed to warp image")
        
    return warped


def showResult(img,gray,threshold,mask,border,corners):
    
    try:
        img = cv.cvtColor(img,cv.COLOR_BGR2RGB)
        fig, ax = plt.subplots(ncols = 2, nrows = 3,figsize=(20,20))
    
        ax[0,0].imshow(img)
        ax[0,0].set_title('Original')

        ax[0,1].imshow(gray,cmap = cm.gray)
        ax[0,1].set_title('Grayscale')

        ax[1,0].imshow(threshold,cmap = cm.gray)
        ax[1,0].set_title('Thresholded')

        ax[1,1].imshow(mask,cmap = cm.gray)
        ax[1,1].set_title('Mask')

        ax[2,0].imshow(border)
        ax[2,0].set_title('Border')

        ax[2,1].imshow(corners)
        ax[2,1].set_title('Corners')
        
    except:
        print("Failed to plot images")

    plt.show

    
def main():
    
    # function to select image
    img = selectImage()
    
    # function to get corner porints
    cornerPoints, threshold, mask, border, corners, gray = getDocumentCorners(img)
    
    #function to sort points
    sorted_pts = sortPoints(cornerPoints)
    
    # function to warp points
    warped = warpPerspective(img,sorted_pts)
    
    # function to plot images
    showResult(img,warped,threshold,mask,border,corners)
    
main()